In [15]:
import pandas as pd
import bs4
from bs4 import BeautifulSoup as bs
import requests
import re

#Check regex

In [20]:
product_data = []

for i in range(1, 7):
    link = f'https://www.ecomdummy.co.nz/search/filter/phones-gps/smart-watches?sf=smart+wearables&fs=26311687&pg={i}#sortGroupForm'
    page = requests.get(link)
    page_content = page.content
    soup = bs(page_content, 'html.parser')

    product_cards = soup.find_all('div', class_='js-product-card')

    for product_card in product_cards:
        # Extract rating, selling price, and number of reviews
        rating_element = product_card.find('span', class_='rating-star')
        reviews_element = product_card.find('span', class_='ratingno')
        price_element = product_card.find('span', class_='font-size-2 fw-bold full-price')

        rating = rating_element['title'] if rating_element else None
        num_reviews = reviews_element.text.strip() if reviews_element else None
        sell_price = price_element.text.strip() if price_element else None

        # Extract data from product-specific link
        product_link = product_card.find('a', class_='js-product-link')
        product_url = product_link['href']
        base_link = "https://www.pbtech.co.nz/"

        product_page = requests.get(base_link + product_url)
        product_soup = bs(product_page.content, 'html.parser')

        brand_element = product_soup.find('span', class_='fw-bold', string='Brand:')
        manufacturer_part_element = product_soup.find('span', class_='fw-bold', string='MPN: ')
        dealer_part_element = product_soup.find('span', class_='fw-bold', string='Part #: ')

        brand_name = brand_element.find_next('span').text.strip() if brand_element else None
        manufacturer_part_number = manufacturer_part_element.find_next('span').text.strip() if manufacturer_part_element else None
        dealer_part_number = dealer_part_element.find_next('span').text.strip() if dealer_part_element else None

        product_info = {
            'Brand Name': brand_name,
            'Manufacturer Part Number': manufacturer_part_number,
            'Dealer Part Number': dealer_part_number,
            "Rating": rating,
            "Number of Reviews": num_reviews,
            "Selling Price": sell_price,
        }

        attributes_section = product_soup.find_all('div', class_='mt-2 product-attributes')

        for attribute_section in attributes_section:
            attribute_name_element = attribute_section.find('p', class_='fw-semibold')
            attribute_value_element = attribute_section.find('p', class_='m-0')

            if attribute_name_element and attribute_value_element:
                attribute_name = attribute_name_element.text.strip()
                attribute_value = attribute_value_element.text.strip()
                product_info[attribute_name] = attribute_value

        spec_divs = product_soup.find_all('div', id="featuresSpecs")

        for spec_div in spec_divs:
            labels = spec_div.find_all('p', class_='label_')
            values = spec_div.find_all('p', class_='value_')

            for label, value in zip(labels, values):
                label_text = label.get_text(strip=True)
                value_text = value.get_text(strip=True)

                if label_text == "Colour":
                    continue

                product_info[label_text] = value_text

        product_data.append(product_info)

# Create a DataFrame from the product_data list
df = pd.DataFrame(product_data)


,Brand Name,Manufacturer Part Number,Dealer Part Number,Rating,Number of Reviews,Selling Price,Model,Size,Compatible OS,Heart Rate Monitor,...,COLOUR,CHARGING PORT,WATER RESISTANCE,Bluetooth,Adjustable length,System language*,General,Packing list,Window Size,Band size
0,Samsung,SM-R900NZDAXNZ,WTHSAM900001,4.3 out of 5 Stars,18,$390.43,Samsung Galaxy Watch5,40mm,Android,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple,MNJP3ZP/A,WTHAPP5623154,5 out of 5 Stars,13,$407.82,Apple Watch SE (2nd gen),40mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple,MNP13ZP/A,WTHAPP5623169,5 out of 5 Stars,20,$677.39,Apple Watch Series 8,45mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Samsung,SM-R900NZAAXNZ,WTHSAM900000,4.5 out of 5 Stars,21,$390.43,Samsung Galaxy Watch5,40mm,Android,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple,MNJT3ZP/A,WTHAPP5623155,5 out of 5 Stars,8,$407.82,Apple Watch SE (2nd gen),40mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Apple,MNK03ZP/A,WTHAPP5623158,4.8 out of 5 Stars,11,$451.30,Apple Watch SE (2nd gen),44mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Apple,MNP63ZP/A,WTHAPP5623173,4.7 out of 5 Stars,6,$633.91,Apple Watch Series 8,41mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Apple,MNP53ZP/A,WTHAPP5623172,4.8 out of 5 Stars,9,$633.91,Apple Watch Series 8,41mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Samsung,SM-R920NZKAXNZ,WTHSAM920000,4.7 out of 5 Stars,11,$651.30,Samsung Galaxy Watch5 Pro,45mm,Android,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Samsung,SM-R860NZKAXNZ,WTHSAM860000,3.8 out of 5 Stars,6,$346.95,Samsung Galaxy Watch4,40mm,Android,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df.head(10)

,Brand Name,Manufacturer Part Number,Dealer Part Number,Rating,Number of Reviews,Selling Price,Model,Size,Compatible OS,Heart Rate Monitor,...,COLOUR,CHARGING PORT,WATER RESISTANCE,Bluetooth,Adjustable length,System language*,General,Packing list,Window Size,Band size
0,Samsung,SM-R900NZDAXNZ,WTHSAM900001,4.3 out of 5 Stars,18,$390.43,Samsung Galaxy Watch5,40mm,Android,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple,MNJP3ZP/A,WTHAPP5623154,5 out of 5 Stars,13,$407.82,Apple Watch SE (2nd gen),40mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple,MNP13ZP/A,WTHAPP5623169,5 out of 5 Stars,20,$677.39,Apple Watch Series 8,45mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Samsung,SM-R900NZAAXNZ,WTHSAM900000,4.5 out of 5 Stars,21,$390.43,Samsung Galaxy Watch5,40mm,Android,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple,MNJT3ZP/A,WTHAPP5623155,5 out of 5 Stars,8,$407.82,Apple Watch SE (2nd gen),40mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Apple,MNK03ZP/A,WTHAPP5623158,4.8 out of 5 Stars,11,$451.30,Apple Watch SE (2nd gen),44mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Apple,MNP63ZP/A,WTHAPP5623173,4.7 out of 5 Stars,6,$633.91,Apple Watch Series 8,41mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Apple,MNP53ZP/A,WTHAPP5623172,4.8 out of 5 Stars,9,$633.91,Apple Watch Series 8,41mm,iOS,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Samsung,SM-R920NZKAXNZ,WTHSAM920000,4.7 out of 5 Stars,11,$651.30,Samsung Galaxy Watch5 Pro,45mm,Android,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Samsung,SM-R860NZKAXNZ,WTHSAM860000,3.8 out of 5 Stars,6,$346.95,Samsung Galaxy Watch4,40mm,Android,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Columns: 127 entries, Brand Name to Band size
dtypes: object(127)
memory usage: 105.3+ KB


In [ ]:
#For single product '

product_data = []

link = f'https://www.eco_dummy.co.nz/search/filter/phones-gps/smart-watches?sf=smart+wearables&fs=26311687&pg={i}#sortGroupForm'
page = requests.get(link)
page_content = page.content
soup = bs(page_content, 'html.parser')

product_cards = soup.find_all('div', class_='js-product-card')

    #for i in product_cards:
    #price_element = i.find('span', class_='font-size-2 fw-bold full-price')
    #sell_price = price_element.text.strip() if price_element else None 
    #print(sell_price)


#product_name_element = soup.find("h2", class_="np_title")
#product_name = product_name_element.text.strip()

#print("Product Name:", product_name)

In [ ]:
df1.head(10)

In [ ]:
df1.info()